In [1]:
# unset PYTHONPATH first
from ko_text import *
from ko_crawler import *

In [2]:
nlp = NLP()

# 불러오기

In [3]:
token_df = pd.read_csv('Data/meta_morphs.csv', encoding = 'utf-8'
                       #converters = {'Token' : ast.literal_eval},
                     )

# 용량을 줄이기 위해 '단어 단어' 꼴로 묶어둔 token을 ['단어', '단어'] 꼴로 풀기
token_df['Token'] = [token.split() if type(token) == str else '' for token in token_df['Token']]

# EDA

# 카테고리 통합

In [7]:
Counter(token_df['Section'])

Counter({'autos': 267,
         'broadcasting': 882,
         'business': 4698,
         'chemistry': 191,
         'culture': 3789,
         'economy': 3873,
         'electronics': 436,
         'entertainment': 129,
         'estate': 4952,
         'financial': 959,
         'health': 1501,
         'heavy_industries': 68,
         'it': 1760,
         'movie': 126,
         nan: 1782,
         'patent': 677,
         'performance': 500,
         'politics': 4824,
         'retail': 1254,
         'society': 4325,
         'stock': 1605,
         'world': 4768})

## nan 카테고리를 ''로 채운다

In [8]:
token_df['Section'].fillna('', inplace = True)

## 카테고리 별로 자주 등장하는 단어 탐색

In [9]:
summary_dict = {'section' : [],
                'keyword_1' : [],
                'keyword_2' : [],
                'keyword_3': [],
                'keyword_4': [],
                'keyword_5': [],
                'count' : [],}

for section in token_df['Section'].unique():
    section_df = token_df[token_df['Section'] == section]

    counter_ls = []
    for tokens in section_df['Token']:
        counter_ls += tokens
    
    counter = Counter(counter_ls).most_common(7)
    
    # 결과 저장
    summary_dict['section'].append(section)
    summary_dict['count'].append(len(section_df))
    for i in range(5):
        summary_dict['keyword_%s'%(i+1)].append(counter[i][0])


In [11]:
pd.DataFrame(summary_dict) 

,count,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,section
0,959,금융,은행,보험,카드,대출,financial
1,3873,기업,미국,중국,올해,경제,economy
2,4952,주택,사업,아파트,분양,지역,estate
3,1605,투자,종목,증권,투자자,탁론,stock
4,4698,기업,사업,기술,시장,제품,business
5,1760,게임,서비스,기술,출시,전자,it
6,436,전자,LG,서비스,KT,기술,electronics
7,677,기술,연구,개발,한국,교수,patent
8,1254,브랜드,상품,제품,롯데,판매,retail
9,1501,치료,환자,병원,바이오,의료,health


In [12]:
# 1. electronics 기사를 it기사로 병합한다.
token_df.loc[token_df['Section'] == 'electronics','Section'] = 'it'

# 2. retail 기사를 business 기사로 병합한다.
token_df.loc[token_df['Section'] == 'retail','Section'] = 'business'

# 3. patent 기사는 bio & tech로 이름을 수정한다.
token_df.loc[token_df['Section'] == 'patent','Section'] = 'bio & tech'

# 4. heavy_industry는 business 기사로 병합한다.
token_df.loc[token_df['Section'] == 'heavy_industries','Section'] = 'business'

# 5. autos는 항공과 관련된 기사, 일관성이 떨어지므로 제거한다.
token_df.drop(token_df[token_df['Section'] == 'autos'].index, inplace = True)

# 6. chemistry는 business 기사로 병합한다.
token_df.loc[token_df['Section'] == 'chemistry','Section'] = 'business'

# 7. nan 기사는 stock 기사로 병합한다.
token_df.loc[token_df['Section'] == '','Section'] = 'stock'

# 8. broadcasting, performance, movie, entertainment은 culture & art 기사로 병합한다.
token_df.loc[token_df['Section'] == 'culture','Section'] = 'culture & art'
token_df.loc[token_df['Section'] == 'broadcasting','Section'] = 'culture & art'
token_df.loc[token_df['Section'] == 'performance','Section'] = 'culture & art'
token_df.loc[token_df['Section'] == 'movie','Section'] = 'culture & art'
token_df.loc[token_df['Section'] == 'entertainment','Section'] = 'culture & art'

# 9. health는 bio & tech 기사로 병합한다.
token_df.loc[token_df['Section'] == 'health','Section'] = 'bio & tech'

# 10.people 기사는 제거
try: token_df.drop(token_df[token_df['Section'] == 'people'].index, inplace = True)
except: pass

In [ ]:
# index도 초기화
token_df.index = np.arange(len(token_df))

In [13]:
Counter(token_df['Section'])

Counter({'bio & tech': 2178,
         'business': 6211,
         'culture & art': 5426,
         'economy': 3873,
         'estate': 4952,
         'financial': 959,
         'it': 2196,
         'politics': 4824,
         'society': 4325,
         'stock': 3387,
         'world': 4768})

In [14]:
# 통합 후, 카테고리의 수가 11개로 감소함
len(set(token_df['Section']))

11

## **제목만 있고 특수 문자를 제거하면 기사 내용이 전혀 없는 뉴스도 존재**

#### 내용이 없는 뉴스들은 제거한다

In [15]:
token_df.shape

(43099, 3)

In [16]:
none_idx_ls = []
for idx, token in enumerate(token_df['Token']):
    if not token:
        none_idx_ls.append(idx)

In [17]:
len(none_idx_ls)

84

In [18]:
Counter(token_df.loc[none_idx_ls, 'Section'])

Counter({'bio & tech': 1,
         'business': 1,
         'culture & art': 59,
         'economy': 4,
         'financial': 2,
         'it': 1,
         'politics': 7,
         'society': 2,
         'stock': 7})

In [19]:
token_df.drop(none_idx_ls, inplace = True)

# 뉴스 기사가 매우 짧은 경우도 제거

In [20]:
token_df['Num of Tokens'] = [len(token) for token in token_df['Token']]

#### 뉴스 기사에 토큰이 하나 밖에 없는 경우도 존재함

In [21]:
token_df['Num of Tokens'].describe()

count    43015.000000
mean       180.184982
std        134.020985
min          1.000000
25%         97.000000
50%        152.000000
75%        229.000000
max       3734.000000
Name: Num of Tokens, dtype: float64

#### 토큰의 수가 30개 이하인 기사는 전체의 3.7% 수준

In [22]:
np.sum([token_df['Num of Tokens'] <= 30]) / len(token_df)

0.037126583749854704

#### 토큰이 30개인 기사의 길이는 대략 1줄 정도

In [23]:
' '.join(token_df.loc[token_df['Num of Tokens'] == 30, 'Token'].iloc[0])

'금융 그룹 글로벌 자본시장 경쟁력 강화 아시아 투자 은행 허브 육성 홍콩 GIB 출범 시켰다 홍콩 현지 열린 출범 용병 금융 회장 이동환 GIB 부문 부행 관계자 테이프 커팅 행사'

#### 토큰의 수가 100개 미만인 기사는 전체의 26% 수준

In [24]:
np.sum([token_df['Num of Tokens'] < 100]) / len(token_df)

0.2607927467162618

## 토큰의 수가 30개 이하인 기사는 표본에서 제거

In [25]:
token_df = token_df[token_df['Num of Tokens'] > 30]

# 결과 저장

### **저장 공간 절약을 위해, ['단어', '단어'] 꼴로 저장된 토큰을 '단어 단어' 꼴로 바꿔준다**

In [26]:
token_df['Token'] = [' '.join(doc) for doc in token_df['Token']]

In [27]:
token_df.to_csv('Data/meta_morphs_final.csv', index = False)